In [32]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

In [33]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')
train

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60
5,5,1991-04-01 00:00:00,0,1,SB,58
6,6,1991-04-01 00:00:00,0,1,WB,26
7,7,1991-04-01 00:00:00,0,2,EB,31
8,8,1991-04-01 00:00:00,0,2,NB,49
9,9,1991-04-01 00:00:00,0,2,SB,46


In [34]:
train['year']=train['time'].map(lambda x: x[0:4]).astype(int)
train['month']=train['time'].map(lambda x: x[5:7]).astype(int)
train['date']=train['time'].map(lambda x: x[8:10]).astype(int)
train['hour']=train['time'].map(lambda x: x[11:13]).astype(int)
train['minute']=train['time'].map(lambda x: x[14:16]).astype(int)
#train[['year','month','date','hour','minute']]
train2=pd.get_dummies(train['direction'])
train=pd.concat([train,train2],axis=1)

In [35]:
test['year']=test['time'].map(lambda x: x[0:4]).astype(int)
test['month']=test['time'].map(lambda x: x[5:7]).astype(int)
test['date']=test['time'].map(lambda x: x[8:10]).astype(int)
test['hour']=test['time'].map(lambda x: x[11:13]).astype(int)
test['minute']=test['time'].map(lambda x: x[14:16]).astype(int)
#train[['year','month','date','hour','minute']]
test2=pd.get_dummies(test['direction'])
test=pd.concat([test,test2],axis=1)
test.describe()

,row_id,x,y,year,month,date,hour,minute,EB,NB,NE,NW,SB,SE,SW,WB
count,2340.000000,2340.000000,2340.000000,2340.0,2340.0,2340.0,2340.00000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000
mean,850004.500000,1.138462,1.630769,1991.0,9.0,30.0,17.50000,20.000000,0.184615,0.184615,0.107692,0.030769,0.184615,0.030769,0.107692,0.169231
std,675.644137,0.801649,1.089611,0.0,0.0,0.0,3.45279,16.333422,0.388068,0.388068,0.310058,0.172729,0.388068,0.172729,0.310058,0.375036
min,848835.000000,0.000000,0.000000,1991.0,9.0,30.0,12.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,849419.750000,0.000000,1.000000,1991.0,9.0,30.0,14.75000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,850004.500000,1.000000,2.000000,1991.0,9.0,30.0,17.50000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,850589.250000,2.000000,3.000000,1991.0,9.0,30.0,20.25000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,851174.000000,2.000000,3.000000,1991.0,9.0,30.0,23.00000,40.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [36]:
train_x=train.drop(['congestion','time','direction','year','minute','row_id'],axis=1)
train_y=train['congestion']
test_x=test.drop(['time','direction','year','minute','row_id'],axis=1)

In [37]:
kf=KFold(n_splits=10,shuffle=True,random_state=71)
tr_idx,va_idx=list(kf.split(train))[0]
tr_x,va_x=train_x.iloc[tr_idx],train_x.iloc[va_idx]
tr_y,va_y=train_y.iloc[tr_idx],train_y.iloc[va_idx]

In [38]:
import xgboost as xgb

In [39]:
dtrain=xgb.DMatrix(tr_x,label=tr_y)
dvalid=xgb.DMatrix(va_x,label=va_y)
dtest=xgb.DMatrix(test_x.values)
dtrain

In [40]:
#params={'objective':'binary:logistic','silent':1,'random_state':71}
params={'objective':'reg:squarederror','random_state':71}
num_round=5000
watchlist=[(dtrain,'train'),(dvalid,'eval')]

In [41]:
model=xgb.train(params,dtrain,num_round,early_stopping_rounds=20,evals=watchlist,)

[0]	train-rmse:36.49034	eval-rmse:36.53777
[1]	train-rmse:27.24983	eval-rmse:27.28398
[2]	train-rmse:21.11478	eval-rmse:21.14206
[3]	train-rmse:17.31627	eval-rmse:17.33381
[4]	train-rmse:14.53182	eval-rmse:14.54239
[5]	train-rmse:13.03633	eval-rmse:13.04252
[6]	train-rmse:12.08865	eval-rmse:12.08760
[7]	train-rmse:11.57676	eval-rmse:11.57304
[8]	train-rmse:11.32346	eval-rmse:11.31973
[9]	train-rmse:10.87978	eval-rmse:10.87808
[10]	train-rmse:10.76956	eval-rmse:10.76658
[11]	train-rmse:10.68646	eval-rmse:10.68256
[12]	train-rmse:10.60251	eval-rmse:10.59484
[13]	train-rmse:10.56742	eval-rmse:10.56077
[14]	train-rmse:10.53087	eval-rmse:10.52421
[15]	train-rmse:10.41908	eval-rmse:10.41726
[16]	train-rmse:10.39861	eval-rmse:10.39713
[17]	train-rmse:10.25347	eval-rmse:10.25399
[18]	train-rmse:10.23269	eval-rmse:10.23391
[19]	train-rmse:10.09002	eval-rmse:10.09341
[20]	train-rmse:10.06628	eval-rmse:10.07038
[21]	train-rmse:10.04796	eval-rmse:10.05123
[22]	train-rmse:10.01442	eval-rmse:10.0188

In [42]:
from sklearn.metrics import mean_squared_error
va_pred=model.predict(dvalid)
score=np.sqrt(mean_squared_error(va_y,va_pred))
print(f'logloss:{score:4f}')

logloss:8.887581


In [43]:
pred=model.predict(xgb.DMatrix(test_x))

In [44]:
pred

array([51.615227, 39.24236 , 52.062336, ..., 65.45723 , 10.877507,
       35.27627 ], dtype=float32)

In [45]:
submit=pd.read_csv('data/sample_submission.csv')
submit['congestion']=pred

In [46]:
submit.to_csv('submission2.csv',index=False)

In [47]:
pred.shape

(2340,)